#  Experimental setup

## Global inputs

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

## Thinking about the algorithm

Question on stackoverflow

  * https://stackoverflow.com/questions/20832994/how-to-seat-everyone-according-to-preferences
  
Similar problem solved in R using a Genetic Algorithm

  * https://github.com/meganstiles/Seating_Chart
  
  1. understand solution
    * Introduction to genetic algorithms https://www.codeproject.com/articles/1104747/introduction-to-genetic-algorithms-with-python-hel
  2. try to translate it into python
  3. expand given algorithm (variable size of guestlist, tables, attributes, translation attributes to closeness matrix)

### Genetic Algorithm based on the tutorial

In [9]:
# write a basic genetic algorithm to make educated guesses on the composition of a string
# 1. implement geneset and target
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!." #possible letters
target = "Hello World!" # solution

# 2. generate a random guess
import random

def generate_parent(length):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

# 3. fitness -> provide feedback -> how well does the guess fit the target
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess)
               if expected == actual)

# 4. mutation
def mutate(parent):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate \
        if newGene == childGenes[index] \
        else newGene
    return ''.join(childGenes)

# 5. feedback
import datetime

def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))
    
    
# main function -> intialise best parent as random sequence
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent)

# loop for iteration over multiple generations always keeping the better fitting one
while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)

    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child


YBO!gbuaZfUX	0	0:00:00.002023
YBO!gbuarfUX	1	0:00:00.003700
YBO!g uarfUX	2	0:00:00.005794
YBO!g uarlUX	3	0:00:00.008408
YBOlg uarlUX	4	0:00:00.009524
YBOlg uorlUX	5	0:00:00.012788
YBllg uorlUX	6	0:00:00.013352
YBllg uorlU!	7	0:00:00.020888
YBllg WorlU!	8	0:00:00.024426
HBllg WorlU!	9	0:00:00.027949
Hellg WorlU!	10	0:00:00.031175
Hellg World!	11	0:00:00.040477
Hello World!	12	0:00:00.042242


# First try with small data set

approach:
  - define guest matrix
  - define binary array with specific proportions (# of 1s = seats per table) -> https://stackoverflow.com/questions/19597473/binary-random-array-with-a-specific-proportion-of-ones
  - "fitness" or "happiness" or "closeness" = (Guestmatrix *x* array_bin) *x* array_bin
  - randomly shuffle array_bin for n iterations compare fitness, keep better one
  - remove picked guests from guestmatrix (delete rows & columns)
  - run again for next table
  - repeat till all guests are seated

In [103]:
# read in test data and transform into an np array for later calculations
guest_matrix =np.array(pd.read_csv("../../data/test_guest.csv", sep=";",usecols= [1,2,3,4,5,6,7,8,9,10]))
guest_matrix

array([[   0, 2000,  200,  200,  200,    0,    0,    0,    0,    0],
       [2000,    0,  200,  200,  200,    0,    0,    0,    0,    0],
       [ 200,  200,    0,  200, 2000,    0,    0,    0,    0,    0],
       [ 200,  200,  200,    0,  200,    0,    0,    0,    0,    0],
       [ 200,  200, 2000,  200,    0,    0,  200,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 2000,  200,  200,    0],
       [   0,    0,    0,    0,  200, 2000,    0,    0,  200,  200],
       [   0,    0,    0,    0,    0,  200,    0,    0,  200,  200],
       [   0,    0,    0,    0,    0,  200,  200,  200,    0, 2000],
       [   0,    0,    0,    0,    0,    0,  200,  200, 2000,    0]])

In [104]:
# creation of binary array based on number of attending people and seats per table
attending = 10
seats_per_table = 5

arr_bin = np.array([1]*seats_per_table + [0]*(attending - seats_per_table) )

#FOR TESTING ONLY !!!
#np.random.shuffle(arr_bin)

# for testing reasons assigned to a variable
arr_1 = np.array(arr_bin)
arr_best = np.array(arr_bin)
fit_best = guest_matrix.dot(arr_best).dot(arr_best)
arr_1

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [24]:
# randomly shuffle array
np.random.shuffle(arr_bin)

In [25]:

arr_2 = arr_bin

In [26]:
arr_1 

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [27]:
arr_2

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0])

In [43]:
# testing calculation logic
guest_matrix.dot(arr_1)

array([2600, 2600, 2600,  800, 2600,    0,  200,    0,    0,    0])

In [41]:
guest_matrix.dot(arr_1).dot(arr_1)

11200

In [44]:
guest_matrix.dot(arr_2)

array([ 200,  200, 2000,  200,  200, 2400, 2400,  400,  600, 2400])

In [42]:
guest_matrix.dot(arr_2).dot(arr_2)

6000

In [106]:
iters = 5

i = 0
while i < iters:
    np.random.shuffle(arr_bin)
    fit_new = guest_matrix.dot(arr_bin).dot(arr_bin)
    
    if fit_new > fit_best:
        arr_best = np.array(arr_bin)
        fit_best = fit_new
    print (arr_best, "; ", fit_best, "; ",arr_bin, "; ",fit_new)
    i += 1

[1 1 1 1 1 0 0 0 0 0] ;  11200 ;  [0 1 1 0 1 0 0 0 1 1] ;  8800
[1 1 1 1 1 0 0 0 0 0] ;  11200 ;  [1 0 0 1 1 0 1 1 0 0] ;  1600
[1 1 1 1 1 0 0 0 0 0] ;  11200 ;  [0 0 1 0 1 1 0 0 1 1] ;  8400
[1 1 1 1 1 0 0 0 0 0] ;  11200 ;  [1 0 0 0 0 1 0 1 1 1] ;  5600
[1 1 1 1 1 0 0 0 0 0] ;  11200 ;  [1 0 1 0 0 1 1 0 0 1] ;  4800


finding all occurences within an array: https://pythonspot.com/array-find/

In [107]:
# determine indices of rows to be deleted 
get_indexes = lambda arr_best, xs: [j for (y, j) in zip(xs, range(len(xs))) if arr_best == y]
i_del = get_indexes(1,arr_best)
print(i_del)

[0, 1, 2, 3, 4]


In [108]:
# deleting seated guests from guest matrix
guest_matrix = np.delete(guest_matrix, i_del, 0)
guest_matrix = np.delete(guest_matrix, i_del, 1)
guest_matrix

array([[   0, 2000,  200,  200,    0],
       [2000,    0,    0,  200,  200],
       [ 200,    0,    0,  200,  200],
       [ 200,  200,  200,    0, 2000],
       [   0,  200,  200, 2000,    0]])

In [68]:
guest_matrix

array([[ 200,  200,    0,  200, 2000,    0,    0,    0,    0,    0],
       [ 200,  200,  200,    0,  200,    0,    0,    0,    0,    0],
       [ 200,  200, 2000,  200,    0,    0,  200,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 2000,  200,  200,    0],
       [   0,    0,    0,    0,  200, 2000,    0,    0,  200,  200],
       [   0,    0,    0,    0,    0,  200,    0,    0,  200,  200],
       [   0,    0,    0,    0,    0,  200,  200,  200,    0, 2000],
       [   0,    0,    0,    0,    0,    0,  200,  200, 2000,    0]])